# K Line - pyecharts
References:
 - https://pyecharts.org/#/
 - https://pyecharts.org/#/zh-cn/rectangular_charts?id=klinecandlestick%ef%bc%9ak%e7%ba%bf%e5%9b%be
 - https://pyecharts.org/#/zh-cn/composite_charts

## 1 Kline

### 1.0 Pre

In [1]:
import pandas as pd
import numpy as np
from pyecharts import options as opts
from pyecharts.charts import Kline, Line
from pyecharts.globals import ThemeType

In [2]:
# input variable
file_name = '中债国债到期收益率：10年.xlsx'
width="900px" # 900
height="500px" # 500

In [3]:
# intermediate variable
label = 'K线'
title = file_name[:-5] + '，K线图'
subtitle = ''

In [4]:
# Prepare data
df_raw = pd.read_excel(file_name)
df_main = df_raw.copy()
df_main.columns = ['date','yield']
df_main = df_main.sort_values('date')
print(df_main.shape)
#df_main.head(3)

(4413, 2)


In [5]:
# open-close generator
def get_open(x):
    return np.nan if x.empty else x.iloc[0]
def get_close(x):
    return np.nan if x.empty else x.iloc[-1]

In [6]:
# plot generator
def calculate_ma(day_count: int, data, obj='close'):
    obj_num = {'open' : 0, 'close' : 1, 'low' : 2, 'high' : 3}[obj]
    result: List[Union[float, str]] = []
    for i in range(len(data)):
        if i < day_count:
            result.append("-")
            continue
        sum_total = 0.0
        for j in range(day_count):
            sum_total += float(data[i - j][obj_num])
        result.append(abs(float("%.3f" % (sum_total / day_count))))
    return result
def kline_base(x_data, data, label, width, height, title='', subtitle='') -> Kline:
    
    init_opts= opts.InitOpts(width=width, height=height)
    
    # data and title
    title_opts = opts.TitleOpts(title=title)
    xaxis_opts = opts.AxisOpts(is_scale=True)
    yaxis_opts = opts.AxisOpts(
                    is_scale=True,
                    splitarea_opts=opts.SplitAreaOpts(
                        is_show=True,
                        areastyle_opts=opts.AreaStyleOpts(opacity=1)))
    
    # color of the candle
    itemstyle_opts = opts.ItemStyleOpts(
        color="#ec0000",
        color0="#00da3c",
        border_color="#8A0000",
        border_color0="#008F28")
    
    # zoom
    #datazoom_opts = [opts.DataZoomOpts(type_="inside")] # zoom inside
    #datazoom_opts = [opts.DataZoomOpts()] # zoom outside
    zoom_opts=[
        opts.DataZoomOpts(
            is_show=False,
            type_="inside",
            range_start=0,
            range_end=100,
        ),
        opts.DataZoomOpts(
            is_show=True,
            type_="slider",
            pos_top="90%",
            range_start=0,
            range_end=100)
    ]
    
    # other
    tooltip_opts=opts.TooltipOpts(
        trigger="axis",
        axis_pointer_type="cross",
        background_color="rgba(245, 245, 245, 0.8)",
        border_width=1,
        border_color="#ccc",
        textstyle_opts=opts.TextStyleOpts(color="#000"),)
    visualmap_opts=opts.VisualMapOpts(
        is_show=False,
        dimension=2,
        series_index=5,
        is_piecewise=True,
        pieces=[
            {"value": 1, "color": "#ec0000"},
            {"value": -1, "color": "#00da3c"},
        ],)
    axispointer_opts=opts.AxisPointerOpts(
        is_show=True,
        link=[{"xAxisIndex": "all"}],
        label=opts.LabelOpts(background_color="#777"),)
    brush_opts=opts.BrushOpts(
        x_axis_index="all",
        brush_link="all",
        out_of_brush={"colorAlpha": 0.1},
        brush_type="lineX",)
    
    # Kline
    kline = Kline(init_opts=init_opts)
    kline.add_xaxis(x_data)
    kline.add_yaxis(label,
                data,
                itemstyle_opts=itemstyle_opts)
    kline.set_global_opts(
        datazoom_opts=zoom_opts,
        title_opts=title_opts,
        yaxis_opts=yaxis_opts,
        xaxis_opts=xaxis_opts,
        tooltip_opts=tooltip_opts,
        visualmap_opts=visualmap_opts,
        axispointer_opts=axispointer_opts,
        brush_opts=brush_opts,
    )
    return kline
def line_base(data, x_data, label, ma_count, obj='close'):
    line = Line()
    line.add_xaxis(xaxis_data=x_data)
    line.add_yaxis(
            series_name=label,
            y_axis=calculate_ma(day_count=ma_count, data=data, obj=obj),
            is_smooth=True,
            is_hover_animation=False,
            linestyle_opts=opts.LineStyleOpts(width=2, opacity=0.6),
            label_opts=opts.LabelOpts(is_show=False),
        )
    line.set_global_opts(xaxis_opts=opts.AxisOpts(type_="category"))
    return line
def generate_Kline(df, file_name, label, width, height):
    df = df.dropna()
    data = df.values.tolist()
    x_data = list(df.index.strftime('%Y/%m/%d'))
    kline = kline_base(x_data, data, label, width, height, title=file_name[:-5], subtitle=subtitle)
    output = kline.overlap(line_base(data, x_data,label='MA5', ma_count=5,  obj='close'))
    output = kline.overlap(line_base(data, x_data,label='MA10', ma_count=10, obj='close'))
    output = kline.overlap(line_base(data, x_data,label='MA20', ma_count=20, obj='close'))
    output.render(file_name)
    return output

### 1.1 Generate OpenCloseLowHigh

In [7]:
# weekly
df_weekly = df_main.groupby(pd.Grouper(key='date', freq='W-SUN')).agg([get_open, get_close, min, max])
df_weekly.columns = ['open','close','low','high']
df_weekly.head(3)

,open,close,low,high
date,,,,
2002-01-06,3.2096,3.2096,3.2096,3.2096
2002-01-13,3.2003,3.4532,3.2003,3.5896
2002-01-20,3.5114,3.2968,3.2968,3.5114


In [8]:
# monthly
df_monthly = df_main.groupby(pd.Grouper(key='date', freq='1M')).agg([get_open, get_close, min, max])
df_monthly.columns = ['open','close','low','high']
df_monthly.head(3)

,open,close,low,high
date,,,,
2002-01-31,3.2096,3.2186,3.0795,3.8100
2002-02-28,3.2016,3.1315,3.1315,3.2491
2002-03-31,3.1155,2.8551,2.8551,3.1155


In [9]:
# seasonally
df_seasonly = df_main.groupby(pd.Grouper(key='date', freq='3M')).agg([get_open, get_close, min, max])
df_seasonly.columns = ['open','close','low','high']
df_seasonly.head(3)

,open,close,low,high
date,,,,
2002-01-31,3.2096,3.2186,3.0795,3.8100
2002-04-30,3.2016,2.5022,1.9775,3.5186
2002-07-31,2.5363,2.8690,2.3353,2.9785


In [10]:
# yearly
df_yearly = df_main.groupby(pd.Grouper(key='date', freq='1Y')).agg([get_open, get_close, min, max])
df_yearly.columns = ['open','close','low','high']
df_yearly.head(3)

,open,close,low,high
date,,,,
2002-12-31,3.2096,3.1776,1.9775,3.8100
2003-12-31,3.0781,3.3602,2.6620,3.9896
2004-12-31,3.1524,4.7332,3.1524,5.4127


### 1.2 output

In [11]:
# output weekly
df = df_weekly
output_file_name = file_name[:-5] + '，周K线.html'
output = generate_Kline(df, output_file_name, label, width, height)
output.render_notebook()

In [12]:
# output monthly
df = df_monthly
output_file_name = file_name[:-5] + '，月K线.html'
output = generate_Kline(df, output_file_name, label, width, height)
output.render_notebook()

In [13]:
# output seasonally
df = df_seasonly
output_file_name = file_name[:-5] + '，季K线.html'
output = generate_Kline(df, output_file_name, label, width, height)
output.render_notebook()

In [14]:
# output yearly
df = df_yearly
output_file_name = file_name[:-5] + '，年K线.html'
output = generate_Kline(df, output_file_name, label, width, height)
output.render_notebook()

## 2 Demo of Grid plot

In [ ]:
from pyecharts import options as opts
from pyecharts.charts import Grid, Kline, Line, Bar
def kline_profession_example() -> Grid:

    data = [
        [2320.26, 2320.26, 2287.3, 2362.94],
        [2300, 2291.3, 2288.26, 2308.38],
        [2295.35, 2346.5, 2295.35, 2345.92],
        [2347.22, 2358.98, 2337.35, 2363.8],
        [2360.75, 2382.48, 2347.89, 2383.76],
        [2383.43, 2385.42, 2371.23, 2391.82],
        [2377.41, 2419.02, 2369.57, 2421.15],
        [2425.92, 2428.15, 2417.58, 2440.38],
        [2411, 2433.13, 2403.3, 2437.42],
        [2432.68, 2334.48, 2427.7, 2441.73],
        [2430.69, 2418.53, 2394.22, 2433.89],
        [2416.62, 2432.4, 2414.4, 2443.03],
        [2441.91, 2421.56, 2418.43, 2444.8],
        [2420.26, 2382.91, 2373.53, 2427.07],
        [2383.49, 2397.18, 2370.61, 2397.94],
        [2378.82, 2325.95, 2309.17, 2378.82],
        [2322.94, 2314.16, 2308.76, 2330.88],
        [2320.62, 2325.82, 2315.01, 2338.78],
        [2313.74, 2293.34, 2289.89, 2340.71],
        [2297.77, 2313.22, 2292.03, 2324.63],
        [2322.32, 2365.59, 2308.92, 2366.16],
        [2364.54, 2359.51, 2330.86, 2369.65],
        [2332.08, 2273.4, 2259.25, 2333.54],
        [2274.81, 2326.31, 2270.1, 2328.14],
        [2333.61, 2347.18, 2321.6, 2351.44],
        [2340.44, 2324.29, 2304.27, 2352.02],
        [2326.42, 2318.61, 2314.59, 2333.67],
        [2314.68, 2310.59, 2296.58, 2320.96],
        [2309.16, 2286.6, 2264.83, 2333.29],
        [2282.17, 2263.97, 2253.25, 2286.33],
        [2255.77, 2270.28, 2253.31, 2276.22],
    ]

    def calculate_ma(day_count: int, d):
        result: List[Union[float, str]] = []
        for i in range(len(d)):
            if i < day_count:
                result.append("-")
                continue
            sum_total = 0.0
            for j in range(day_count):
                sum_total += float(d[i - j][1])
            result.append(abs(float("%.3f" % (sum_total / day_count))))
        return result

    x_data = ["2017-7-{}".format(i + 1) for i in range(31)]

    kline = (
        Kline()
        .add_xaxis(xaxis_data=x_data)
        .add_yaxis(
            series_name="Dow-Jones index",
            y_axis=data,
            itemstyle_opts=opts.ItemStyleOpts(color="#ec0000", color0="#00da3c"),
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(
                title="复杂版 Kline 示例 (Kline + Line + Bar)",
                subtitle="MA 就以(2, 4, 6, 8为例)",
            ),
            xaxis_opts=opts.AxisOpts(type_="category"),
            yaxis_opts=opts.AxisOpts(
                is_scale=True,
                splitarea_opts=opts.SplitAreaOpts(
                    is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
                ),
            ),
            legend_opts=opts.LegendOpts(
                is_show=False, pos_bottom=10, pos_left="center"
            ),
            datazoom_opts=[
                opts.DataZoomOpts(
                    is_show=False,
                    type_="inside",
                    xaxis_index=[0, 1],
                    range_start=0,
                    range_end=100,
                ),
                opts.DataZoomOpts(
                    is_show=True,
                    xaxis_index=[0, 1],
                    type_="slider",
                    pos_top="90%",
                    range_start=0,
                    range_end=100,
                ),
            ],
            tooltip_opts=opts.TooltipOpts(
                trigger="axis",
                axis_pointer_type="cross",
                background_color="rgba(245, 245, 245, 0.8)",
                border_width=1,
                border_color="#ccc",
                textstyle_opts=opts.TextStyleOpts(color="#000"),
            ),
            visualmap_opts=opts.VisualMapOpts(
                is_show=False,
                dimension=2,
                series_index=5,
                is_piecewise=True,
                pieces=[
                    {"value": 1, "color": "#ec0000"},
                    {"value": -1, "color": "#00da3c"},
                ],
            ),
            axispointer_opts=opts.AxisPointerOpts(
                is_show=True,
                link=[{"xAxisIndex": "all"}],
                label=opts.LabelOpts(background_color="#777"),
            ),
            brush_opts=opts.BrushOpts(
                x_axis_index="all",
                brush_link="all",
                out_of_brush={"colorAlpha": 0.1},
                brush_type="lineX",
            ),
        )
    )

    line = (
        Line()
        .add_xaxis(xaxis_data=x_data)
        .add_yaxis(
            series_name="MA2",
            y_axis=calculate_ma(day_count=2, d=data),
            is_smooth=True,
            is_hover_animation=False,
            linestyle_opts=opts.LineStyleOpts(width=3, opacity=0.5),
            label_opts=opts.LabelOpts(is_show=False),
        )
        .add_yaxis(
            series_name="MA4",
            y_axis=calculate_ma(day_count=4, d=data),
            is_smooth=True,
            is_hover_animation=False,
            linestyle_opts=opts.LineStyleOpts(width=3, opacity=0.5),
            label_opts=opts.LabelOpts(is_show=False),
        )
        .add_yaxis(
            series_name="MA6",
            y_axis=calculate_ma(day_count=6, d=data),
            is_smooth=True,
            is_hover_animation=False,
            linestyle_opts=opts.LineStyleOpts(width=3, opacity=0.5),
            label_opts=opts.LabelOpts(is_show=False),
        )
        .add_yaxis(
            series_name="MA8",
            y_axis=calculate_ma(day_count=8, d=data),
            is_smooth=True,
            is_hover_animation=False,
            linestyle_opts=opts.LineStyleOpts(width=3, opacity=0.5),
            label_opts=opts.LabelOpts(is_show=False),
        )
        .set_global_opts(xaxis_opts=opts.AxisOpts(type_="category"))
    )

    bar = (
        Bar()
        .add_xaxis(xaxis_data=x_data)
        .add_yaxis(
            series_name="Volume",
            yaxis_data=[
                [i, data[i][3], 1 if data[i][0] > data[i][1] else -1]
                for i in range(len(data))
            ],
            xaxis_index=1,
            yaxis_index=1,
            label_opts=opts.LabelOpts(is_show=False),
        )
        .set_global_opts(
            xaxis_opts=opts.AxisOpts(
                type_="category",
                is_scale=True,
                grid_index=1,
                boundary_gap=False,
                axisline_opts=opts.AxisLineOpts(is_on_zero=False),
                axistick_opts=opts.AxisTickOpts(is_show=False),
                splitline_opts=opts.SplitLineOpts(is_show=False),
                axislabel_opts=opts.LabelOpts(is_show=False),
                split_number=20,
                min_="dataMin",
                max_="dataMax",
            ),
            yaxis_opts=opts.AxisOpts(
                grid_index=1,
                is_scale=True,
                split_number=2,
                axislabel_opts=opts.LabelOpts(is_show=False),
                axisline_opts=opts.AxisLineOpts(is_show=False),
                axistick_opts=opts.AxisTickOpts(is_show=False),
                splitline_opts=opts.SplitLineOpts(is_show=False),
            ),
            legend_opts=opts.LegendOpts(is_show=False),
        )
    )

    # Kline And Line
    overlap_kline_line = kline.overlap(line)

    # Grid Overlap + Bar
    grid_chart = Grid()
    grid_chart.add(
        overlap_kline_line,
        grid_opts=opts.GridOpts(pos_left="10%", pos_right="8%", height="50%"),
    )
    grid_chart.add(
        bar,
        grid_opts=opts.GridOpts(
            pos_left="10%", pos_right="8%", pos_top="70%", height="16%"
        ),
    )
    return grid_chart
#c1 = kline_profession_example()
#c1.render_notebook()